In [1]:
import yaml
import h5py
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from loguru import logger as lg
from termcolor import colored

### Open conf file

In [14]:
with open("../config/sed_16k.yaml", "r") as f:
    conf = yaml.safe_load(f)

In [15]:
conf["data"]

{'audio_max_len': 10,
 'fs': 16000,
 'net_subsample': 4,
 'root_path': '/home/tianzichen/SELDProjects/CoSMo/data/',
 'hdf5_train': 'train_16k.h5',
 'hdf5_val': 'val_16k.h5',
 'hdf5_test': 'test_16k.h5',
 'sonyc_csv_train': 'metadata/train/SONYC_train.csv',
 'sonyc_csv_val': 'metadata/val/SONYC_val.csv',
 'sonyc_csv_test': 'metadata/test/SONYC_test.csv',
 'raw_singa-pura_csv_root': '/home/tianzichen/sound_datasets/singapura/labels_public',
 'singa-pura_csv_train': 'metadata/train/SINGA-PURA_train.csv',
 'singa-pura_csv_val': 'metadata/val/SINGA-PURA_val.csv',
 'singa-pura_csv_test': 'metadata/test/SINGA-PURA_test.csv',
 'singa-pura_csv_unlabelled': 'metadata/train/SINGA-PURA_unlabelled.csv',
 'taxonomy_path': '/home/tianzichen/SELDProjects/CoSMo/config/taxonomy_SINGA-PURA.yaml'}

In [4]:
def print_hf(hf_path):
    hf = h5py.File(hf_path)

    def recursive_print(hf, tab):
        for name in hf:
            if isinstance(hf[name], h5py.Group):
                print("".join(["\t"] * tab) + name)
                recursive_print(hf[name], tab + 1)
            else:
                print(
                    "".join(["\t"] * tab) + str(hf[name]).replace("<HDF5 dataset", "")
                )

    recursive_print(hf, 0)

In [18]:
print_hf("../data/val_16k.h5")

SINGA-PURA
	 "audio_32k": shape (736, 160000), type "<f4">
	 "filenames": shape (736,), type "|O">
	groundtruth
		 "taxonomy_coarse_SINGA-PURA": shape (736, 15, 78), type "<f8">
		 "taxonomy_coarse_SONYC": shape (736, 8, 78), type "<f8">
		 "taxonomy_fine_SINGA-PURA": shape (736, 50, 78), type "<f8">
		 "taxonomy_fine_SONYC": shape (736, 29, 78), type "<f8">
SONYC
	 "audio_32k": shape (538, 160000), type "<f4">
	 "filenames": shape (538,), type "|O">
	groundtruth
		 "taxonomy_coarse_SINGA-PURA": shape (538, 15), type "<f8">
		 "taxonomy_coarse_SONYC": shape (538, 8), type "<f8">
		 "taxonomy_fine_SINGA-PURA": shape (538, 50), type "<f8">
		 "taxonomy_fine_SONYC": shape (538, 29), type "<f8">


In [ ]:
print_hf("../data/train.h5")

SINGA-PURA
	 "audio_32k": shape (4435, 160000), type "<f4">
	 "filenames": shape (4435,), type "|O">
	groundtruth
		 "taxonomy_coarse_SINGA-PURA": shape (4435, 15, 78), type "<f8">
		 "taxonomy_coarse_SONYC": shape (4435, 8, 78), type "<f8">
		 "taxonomy_fine_SINGA-PURA": shape (4435, 50, 78), type "<f8">
		 "taxonomy_fine_SONYC": shape (4435, 29, 78), type "<f8">
SONYC
	 "audio_32k": shape (12978, 160000), type "<f4">
	 "filenames": shape (12978,), type "|O">
	groundtruth
		 "taxonomy_coarse_SINGA-PURA": shape (12978, 15), type "<f8">
		 "taxonomy_coarse_SONYC": shape (12978, 8), type "<f8">
		 "taxonomy_fine_SINGA-PURA": shape (12978, 50), type "<f8">
		 "taxonomy_fine_SONYC": shape (12978, 29), type "<f8">
unlabelled_SINGA-PURA
	 "audio_32k": shape (30001, 160000), type "<f4">
	 "filenames": shape (30001,), type "|O">


In [20]:
print_hf(os.path.join(conf["data"]["root_path"], "test_16k.h5"))

SINGA-PURA
	 "audio_32k": shape (1376, 160000), type "<f4">
	 "filenames": shape (1376,), type "|O">
	groundtruth
		 "taxonomy_coarse_SINGA-PURA": shape (1376, 15, 78), type "<f8">
		 "taxonomy_coarse_SONYC": shape (1376, 8, 78), type "<f8">
		 "taxonomy_fine_SINGA-PURA": shape (1376, 50, 78), type "<f8">
		 "taxonomy_fine_SONYC": shape (1376, 29, 78), type "<f8">
SONYC
	 "audio_32k": shape (664, 160000), type "<f4">
	 "filenames": shape (664,), type "|O">
	groundtruth
		 "taxonomy_coarse_SINGA-PURA": shape (664, 15), type "<f8">
		 "taxonomy_coarse_SONYC": shape (664, 8), type "<f8">
		 "taxonomy_fine_SINGA-PURA": shape (664, 50), type "<f8">
		 "taxonomy_fine_SONYC": shape (664, 29), type "<f8">


In [6]:
hf.close()

NameError: name 'hf' is not defined

In [ ]:
@lg.catch
def clone_hdf5(hdf5_file_path, new_hdf5_file_path, ignore_dataset_names=[]):
    hdf5_file = h5py.File(hdf5_file_path, "r")
    new_hdf5_file = h5py.File(new_hdf5_file_path, "w-")

    def recursive_clone(src, tgt, ignore_dataset_names, tab=0):
        for name in src:
            if isinstance(src[name], h5py.Group):
                print(colored("".join(["\t"] * tab) + name, attrs=["bold"]))
                tgt.create_group(name)
                recursive_clone(src[name], tgt[name], ignore_dataset_names, tab + 1)
            else:
                print(
                    colored(
                        "".join(["\t"] * tab)
                        + str(src[name]).replace("<HDF5 dataset", ""),
                        "red",
                    )
                )
                if name not in ignore_dataset_names:
                    src.copy(src[name], tgt)

    recursive_clone(hdf5_file, new_hdf5_file, ignore_dataset_names)

    hdf5_file.close()
    new_hdf5_file.close()

In [ ]:
root_path = conf["data"]["root_path"]
cloned_path = os.path.join(root_path, "cloned")
to_clone = "test.h5"

In [ ]:
ignore_dataset_names = ["open_l3_6144", "open_l3_512", "audio_32k", "passt_2levelmel"]
clone_hdf5(
    os.path.join(root_path, to_clone),
    os.path.join(cloned_path, to_clone),
    ignore_dataset_names,
)

2025-03-25 21:42:07.190 | ERROR    | __main__:<cell line: 2>:2 - An error has been caught in function '<cell line: 2>', process 'MainProcess' (6367), thread 'MainThread' (140637101754176):
Traceback (most recent call last):

  File "/home/tianzichen/anaconda3/envs/CoSMo/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x7fe89f148b30, file "/home/tianzichen/anaconda3/envs/CoSMo/lib/python3.8/site-packages/ipykernel_la...
           └ <function _run_code at 0x7fe89f155ca0>
  File "/home/tianzichen/anaconda3/envs/CoSMo/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pac

In [ ]:
import openl3
import numpy as np
from tqdm import tqdm


def compute_l3_embeddings(hf_path):
    hf = h5py.File(hf_path, "r+")
    model = openl3.models.load_audio_embedding_model(
        input_repr="mel256", content_type="env", embedding_size=512, frontend="kapre"
    )
    if "train_hear.h5" in hf_path:
        hf_audio = h5py.File(hf_path.replace("_hear", ""), "r")
    else:
        hf_audio = hf

    for dname in ["SONYC", "SINGA-PURA"]:
        print(dname)
        audio_dset = hf_audio[dname]["audio_32k"]
        dset_size = audio_dset.shape[0]
        embed_dset = hf[dname].require_dataset(
            "open_l3_512", shape=(dset_size, 96, 512), dtype="float32"
        )
        for i in tqdm(range(dset_size)):
            embed_dset[i], _ = openl3.get_audio_embedding(
                audio_dset[i], 32000, model=model, batch_size=96, verbose=False
            )

In [ ]:
for hf_name in ["train_hear.h5", "val.h5", "test.h5"]:
    print(hf_name)
    compute_l3_embeddings(os.path.join(root_path, hf_name))

train_hear.h5


NameError: name 'compute_l3_embeddings' is not defined